## 아래 실습은 Colab에서 하기를 권장
- Docker 환경에서는 에러가 발생

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845500 sha256=d2f06199ead0955d3890af78f1c81e499648ab67a098d8af03f6decc45e3f952
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


## 실습 5. Redshift 연결해보기

- MAU (Monthly Active User) 계산해보기
-  두 개의 테이블을 Redshift에서 Spark으로 로드
    -  JDBC 연결 실습
-  DataFrame과 SparkSQL을 사용해서 조인
-  DataFrame JOIN
    -  left_DF.join(right_DF, join condition, join type)
        -  join type: “inner”, “left”, “right”, “outer”, “semi”, “anti”

In [30]:
# !sudo mkdir -p /usr/local/lib/python3.10/dist-packages/pyspark/jars/

In [2]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && sudo wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar

--2023-07-04 17:37:21--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.54.128, 52.217.170.200, 52.217.126.128, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.54.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048241 (1024K) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.14.jar’

redshift-jdbc42-2.1 100%[===================>]   1024K  2.33MB/s    in 0.4s    

2023-07-04 17:37:22 (2.33 MB/s) - ‘redshift-jdbc42-2.1.0.14.jar’ saved [1048241/1048241]



In [ ]:
!ls /usr/local/lib/python3.10/dist-packages/pyspark/jars/

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame #5") \
    .getOrCreate()

In [7]:
spark.stop()

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame #5") \
    .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/redshift-jdbc42-2.1.0.14.jar") \
    .getOrCreate()

In [9]:
spark

## Redshift와 연결해서 테이블들을 데이터프레임으로 로딩하기

In [10]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [11]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [12]:
df_user_session_channel.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)



In [13]:
df_user_session_channel.rdd.getNumPartitions()

1

In [14]:
df_session_timestamp.printSchema()

root
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [15]:
df_session_timestamp.rdd.getNumPartitions()

1

## DataFrame으로 처리하기

In [16]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [17]:
session_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [18]:
session_df.show(5)

+------+--------------------+-------+--------------------+--------------------+
|userid|           sessionid|channel|           sessionid|                  ts|
+------+--------------------+-------+--------------------+--------------------+
|   243|0226aa5193c66d990...| Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|   939|02b8d6c2775b756de...| Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
|   771|02f6364bf207d6837...|  Naver|02f6364bf207d6837...|2019-09-07 18:24:...|
|  1961|0302915889fa38fe5...|Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
|  1953|0472919f6bb6a2f7c...|Organic|0472919f6bb6a2f7c...|2019-09-25 06:44:...|
+------+--------------------+-------+--------------------+--------------------+
only showing top 5 rows



In [20]:
session_df = df_user_session_channel.alias('a').join(
    df_session_timestamp.alias('b'),
    join_expr,
    "inner"
).select("a.userid", "a.sessionid", "a.channel", "b.ts")

# dataframe에 alias를 붙이거나 아래처럼 어느 테이블에서 나온건지 명시

In [21]:
# dataframe에 alias를 붙이거나 아래처럼 어느 테이블에서 나온건지 명시

session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", df_user_session_channel.sessionid, "channel", "ts"
)

In [22]:
channel_count_df = session_df.groupby("channel").count().orderBy("count", ascending=False)

In [23]:
channel_count_df.show()

+---------+-----+
|  channel|count|
+---------+-----+
|  Youtube|17091|
|   Google|16982|
|    Naver|16921|
|  Organic|16904|
|Instagram|16831|
| Facebook|16791|
+---------+-----+



In [24]:
from pyspark.sql.functions import date_format, asc, countDistinct

session_df.withColumn('month', date_format('ts', 'yyyy-MM')).groupby('month').\
    agg(countDistinct("userid").alias("mau")).sort(asc('month')).show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-08|662|
|2019-09|639|
|2019-10|763|
|2019-11|721|
+-------+---+



## Spark SQL로  처리하기

In [25]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")

In [26]:
df_session_timestamp.createOrReplaceTempView("session_timestamp")

In [27]:
channel_count_df = spark.sql("""
    SELECT channel, count(distinct userId) uniqueUsers
    FROM session_timestamp st
    JOIN user_session_channel usc ON st.sessionID = usc.sessionID
    GROUP BY 1
    ORDER BY 1
""")

In [28]:
channel_count_df

DataFrame[channel: string, uniqueUsers: bigint]

In [29]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+



In [30]:
mau_df = spark.sql("""
SELECT
  LEFT(A.ts, 7) AS month,
  COUNT(DISTINCT B.userid) AS mau
FROM session_timestamp A
JOIN user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC""")

In [31]:
mau_df.collect()

[Row(month='2019-11', mau=721),
 Row(month='2019-10', mau=763),
 Row(month='2019-09', mau=639),
 Row(month='2019-08', mau=662),
 Row(month='2019-07', mau=623),
 Row(month='2019-06', mau=459),
 Row(month='2019-05', mau=281)]